In [1]:
import numpy as np
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaModel
from torch.optim import Adam, lr_scheduler
import torch.nn.functional as F
import sys
sys.path.append('../')

import os
import argparse
import json
import torch.nn as nn

from util import *
from losses import LabelSmoothingCrossEntropy, SupConLoss
from augment import *

from torch.utils.data.dataset import ConcatDataset
# from torch_model import SupConRobertaNet, SupConMultiRobertaNet
from torch.utils.data.sampler import RandomSampler
from torch_model import TransferRobertaNet, ContraRobertaNet
from feed import *
from torchsampler import ImbalancedDatasetSampler
from losses import FocalLoss
from variables import *

/home/beomgon2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/beomgon2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/beomgon2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/beomgon2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [2]:
BATCH_SIZE = 12
MAX_SEQ_LEN = 512

train_df = pd.read_csv('files/train2.csv')
test_df = pd.read_csv('files/test2.csv')
# train_df = pd.read_csv('data/files/disease_train.csv')
# test_df = pd.read_csv('data/files/disease_test.csv')
Num_Label = len(train_df.label_id.value_counts())
print(Num_Label)
print(train_df.shape)
print(test_df.shape)

10
(4280, 5)
(476, 5)


In [3]:
# train_df.label_id.value_counts()

In [4]:
# alpha = (1/(train_df.label_id.value_counts()/1631))**0.5
# alpha = alpha.to_dict()
# print(alpha)
# def f1(x) :
#     return x[0]
# alpha = sorted(alpha.items(), key=f1, reverse=False)
# print(alpha)
# alpha = [ v for l, v in alpha]
# print(alpha)
# alpha = torch.tensor(alpha)
# print(alpha)

In [5]:
df_dict ={}
train_df = train_df[['SE', 'label_id']]
import sys
for label in range(Num_Label) :
    df = train_df[train_df['label_id'] == label]
    df_dict[label] = df
#     print(label, ' : ', len(df))

In [6]:
device = torch.device("cuda")
# device = torch.device('cpu')
pretrained_path = './pretrained_0818/'
tokenizer = RobertaTokenizer.from_pretrained(pretrained_path, do_lower_case=False)
model = TransferRobertaNet(path=pretrained_path,                       
                              embedding_dim=768,
                              num_class=10,
                              num_class1=10)
model.to(device)
# alpha.to(device)

TransferRobertaNet(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [7]:
# model_clone = TransferRobertaNet(path=pretrained_path,                       
#                               embedding_dim=768,
#                               num_class=400,
#                               num_class1=10)
# model_clone.to(device)

In [8]:
def reset_model(model, model_state=None) :
    if model_state is not None :
        model.load_state_dict(model_state)

#     return model

In [9]:
train_dataset = PetDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
# indices = list(range(len(train_dataset)))
# num_samples = len(indices)
# train_loader = torch.utils.data.DataLoader(
#     train_dataset, 
#     sampler=ImbalancedDatasetSampler(train_df, indices=indices, num_samples=num_samples),
#     batch_size=12, 
#     shuffle=False, 
#     num_workers=2
# )

In [10]:
optimizer = Adam(model.parameters(), lr=0.00008)

scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / ((epoch/4) + 1)
)

In [11]:
def model_eval(test_df, model, istransfer=True) :
    model.eval()

    test_dataset = PetDataset(test_df)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

    total_loss = 0
    total_len = 0
    total_correct = 0

    for text, label in test_loader:
        #   encoded_list = [tokenizer.encode(t, add_special_token=True) for t in text]
          encoded_list = [tokenizer.encode(t, max_length=512, truncation=True) for t in text]
          padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
          sample = torch.tensor(padded_list)
          sample, label = sample.to(device), label.to(device)
          labels = torch.tensor(label)
          outputs = model(sample=sample, istransfer=istransfer)

          pred = torch.argmax(F.softmax(outputs), dim=1)
          correct = pred.eq(labels)
          total_correct += correct.sum().item()
          total_len += len(labels)

    print('Test accuracy: ', total_correct / total_len)
    return total_correct / total_len

In [12]:
# model_eval(test_df, model, istransfer=True)

In [13]:
# # reptile version 1, equal to joint training, gradient update one times in inner loop
# criterion = FocalLoss(alpha=0.97, reduce=True)
# criterion = criterion.to(device)

# epochs = 15
# model.train()
# high_acc = 0
# task_batch_size = 10
# for epoch in range(epochs):
#     losses = AverageMeter()
#     total_loss = 0
#     total_len = 0
#     total_correct = 0
#     total_count = 0
#     model.train()
#     updates = []
#     model_backup = model.state_dict()
#     for text, label in train_loader:
#         reset_model(model, model_backup)

#         encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=MAX_SEQ_LEN, truncation=True) for t in text]
#         padded_list = [e[:MAX_SEQ_LEN] + [0] * (MAX_SEQ_LEN-len(e[:MAX_SEQ_LEN])) for e in encoded_list]
#         sample = torch.tensor(padded_list)
#         sample, label = sample.to(device), label.to(device)
#         labels = torch.tensor(label)
#         outputs = model(sample=sample, istransfer=True)
# #         print(outputs.size())
# #         loss, logits = outputs

#         loss = criterion(outputs, label)
#         losses.update(loss.item(), BATCH_SIZE)

#         pred = torch.argmax(F.softmax(outputs), dim=1)
#         correct = pred.eq(labels)
#         total_correct += correct.sum().item()
#         total_len += len(labels)
#         total_loss += loss.item()
#         total_count += 1
        
#         if (total_count) % 2000 == 0:
#             print('Train: [{0}][{1}]\t'
#                   'loss {loss.avg:.5f}'.format(
#                    epoch, total_count, loss=losses))  

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         print(total_count)
        
#         updates.append(subtract_vars(model.state_dict(), model_backup))
#         if total_count % task_batch_size == 0 :
#             update = average_vars(updates)
#             updates = []
#             model_backup = add_vars(model_backup, scale_vars(update, epsilon=0.99))
        
#     scheduler.step()
#     accr = model_eval(test_df, model, istransfer=True)
#     if accr > high_acc :
#         high_acc = accr
#         torch.save(model.state_dict(), 'maml/transfer')
#         print('model is saved')

#     print('[Epoch {}/{}] Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, epochs, total_loss / total_count, total_correct / total_len))


In [14]:
# def get_text(labels, df_dict) :    
#     seed = np.random.randint(0, 100000, size=None)
#     df_list = []
#     for label in labels.numpy() :
#         df_train = df_dict[label].sample(n=1, random_state=seed, replace=True )
#         df_list.append(df_train)
#     df = pd.concat(df_list)
# #     text1 = df.SE
#     return df.SE

In [15]:
# # reptile version 2, gradient update three times in inner loop
# criterion = FocalLoss(alpha=0.97, reduce=True)
# criterion = criterion.to(device)

# epochs = 15
# model.train()
# high_acc = 0
# task_batch_size = 10
# # label_list = list(train_df.label_id.value_counts().to_dict().keys())

# for epoch in range(epochs):
#     losses = AverageMeter()
#     total_loss = 0
#     total_len = 0
#     total_correct = 0
#     total_count = 0
#     model.train()
#     updates = []
#     op_updates = []
#     model_backup = model.state_dict()
#     op_state = optimizer.state_dict()
#     for text, labels in train_loader:
#         reset_model(model, model_backup)
#         optimizer.load_state_dict(op_state)
        
#         text1 = get_text(labels, df_dict )
#         text2 = get_text(labels, df_dict )
        
#         for text in [text, text1, text2] :
#             encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=MAX_SEQ_LEN, truncation=True) for t in text]
#             padded_list = [e[:MAX_SEQ_LEN] + [0] * (MAX_SEQ_LEN-len(e[:MAX_SEQ_LEN])) for e in encoded_list]
#             sample = torch.tensor(padded_list)
#             sample, labels = sample.to(device), labels.to(device)
#             labels = torch.tensor(labels)
#             outputs = model(sample=sample, istransfer=True)

#             loss = criterion(outputs, labels)
#             losses.update(loss.item(), BATCH_SIZE)

#             pred = torch.argmax(F.softmax(outputs), dim=1)
#             correct = pred.eq(labels)
#             total_correct += correct.sum().item()
#             total_len += len(labels)
#             total_loss += loss.item()
#             total_count += 1
        
#             if (total_count) % 2000 == 0:
#                 print('Train: [{0}][{1}]\t'
#                       'loss {loss.avg:.5f}'.format(
#                        epoch, total_count, loss=losses))  

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
        
#         updates.append(subtract_vars(model.state_dict(), model_backup))
#         op_state = optimizer.state_dict()
#         if total_count % task_batch_size == 0 :
#             update = average_vars(updates)
#             updates = []
#             model_backup = add_vars(model_backup, scale_vars(update, epsilon=0.99))
       
#     scheduler.step()
#     accr = model_eval(test_df, model, istransfer=True)
#     if accr > high_acc :
#         high_acc = accr
#         torch.save(model.state_dict(), 'maml/transfer1')
#         print('model is saved')

#     print('[Epoch {}/{}] Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, epochs, total_loss / total_count, total_correct / total_len))


In [16]:
# optimizer.state_dict()

In [17]:
# !nvidia-smi

In [18]:
# model.load_state_dict(torch.load('maml/transfer'))

In [19]:
# train_df = pd.read_csv('files/train2.csv')
# test_df = pd.read_csv('files/test2.csv')
# train_df = pd.read_csv('data/files/disease_train.csv')
# test_df = pd.read_csv('data/files/disease_test.csv')
train_df = pd.read_csv('files/train2.csv')
test_df = pd.read_csv('files/test2.csv')
Label_num = len(train_df.label_id.value_counts())
print(Label_num)
print(train_df.shape)
print(test_df.shape)

10
(4280, 5)
(476, 5)


In [20]:
train_dataset = PetDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=2)

In [21]:
optimizer = Adam(model.parameters(), lr=0.00002)

scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / ((epoch/4) + 1)
)

In [22]:
# for param, state in zip(model.parameters(), model.state_dict()) :
#     if 'down_fc.' not in state :
#         param.requires_grad = False
#     else :
#         param.requires_grad = True

In [23]:
# criterion = FocalLoss(alpha=alpha,gamma=2, reduce=True)

In [24]:
criterion = FocalLoss(alpha=0.97, gamma=2, reduce=True)
# criterion = FocalLoss(alpha=alpha, gamma=2, reduce=True)
# criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)

epochs = 10
model.train()
high_acc = 0
for epoch in range(epochs):
    total_loss = 0
    total_len = 0
    total_correct = 0
    total_count = 0
    model.train()
    for text, label in train_loader:
#         print(label)

        encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=512, truncation=True) for t in text]
        padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample=sample, istransfer=False)
#         print(outputs.size())
#         loss, logits = outputs

        loss = criterion(outputs, label)

        pred = torch.argmax(F.softmax(outputs), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        total_count += 1
        if total_count % 2000 == 0 :
            print('loss ', total_loss/total_count)

        optimizer.zero_grad()    
        loss.backward()
        optimizer.step()
    scheduler.step()
    accr = model_eval(test_df, model, istransfer=False)
    if accr > high_acc :
        high_acc = accr
        torch.save(model.state_dict(), 'tune1')

    print('[Epoch {}/{}] Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, epochs, total_loss / total_count, total_correct / total_len))
# torch.save(model.state_dict(), 'pretrain')    

/home/beomgon2/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/beomgon2/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/home/beomgon2/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/beomgon2/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/beomgon2/.local/lib/python3.6/site-packa

Test accuracy:  0.7794117647058824
[Epoch 1/10] Train Loss: 0.7978, Accuracy: 0.637
Test accuracy:  0.792016806722689
[Epoch 2/10] Train Loss: 0.3584, Accuracy: 0.812
Test accuracy:  0.819327731092437
[Epoch 3/10] Train Loss: 0.2491, Accuracy: 0.869
Test accuracy:  0.819327731092437
[Epoch 4/10] Train Loss: 0.1763, Accuracy: 0.910
Test accuracy:  0.8214285714285714
[Epoch 5/10] Train Loss: 0.1229, Accuracy: 0.937
Test accuracy:  0.8130252100840336
[Epoch 6/10] Train Loss: 0.0833, Accuracy: 0.964
Test accuracy:  0.8235294117647058
[Epoch 7/10] Train Loss: 0.0631, Accuracy: 0.974
Test accuracy:  0.8004201680672269
[Epoch 8/10] Train Loss: 0.0448, Accuracy: 0.983
Test accuracy:  0.8067226890756303
[Epoch 9/10] Train Loss: 0.0334, Accuracy: 0.986


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/beomgon2/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-a749efc0ce64>", line 39, in <module>
    loss.backward()
  File "/home/beomgon2/.local/lib/python3.6/site-packages/torch/tensor.py", line 185, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/beomgon2/.local/lib/python3.6/site-packages/torch/autograd/__init__.py", line 127, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/beomgon2/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling 

TypeError: object of type 'NoneType' has no len()